Methodology was based on Section 6.2 of paper "Quantifying measurement quality and load distribution in Tor": https://dl.acm.org/doi/pdf/10.1145/3427228.3427238

* I can always create a new container image with a Tor process representing each client and it will always choose a new guard node, so I don't have to change the default Tor definitions
* Log all relays along with their data, specifically geographical location, used in each client circuit.
* In the paper, the authors logged over 8.6 million circuits, representing about 275,000 circuits per day
* Drop the circuit immediately after creation to ensure not overloading the guard relays
* Wait 16 minutes between each circuit

In [ ]:
!pip install stem
!pip install retrying
!pip install flask


In [ ]:
import subprocess
import requests
import shlex
import traceback
import time
import threading
from stem import CircStatus
from stem.control import Controller
import stem.process
import stem.descriptor.remote
from retrying import retry
import os
import shutil
from flask import Flask
import sys


In [ ]:
BASE_DIR = 'data'
SOCKS_PORT = 9050
CONTROL_PORT = 9051
ONION_SERVICE_FOLDER = 'onion_service/'
WEB_PORT = 8080

In [ ]:
def stop_tor():
    command = "sudo pkill tor"
    tor_process = subprocess.Popen(shlex.split(command))
    return tor_process

In [ ]:
def start_tor():
    command = "/opt/homebrew/opt/tor/bin/tor -f data/torrc"
    tor_process = subprocess.Popen(shlex.split(command))
    return tor_process

### More stem references
* https://tor.stackexchange.com/questions/7049/stem-how-to-get-current-in-use-circuit
* https://github.com/webfp/tor-browser-selenium/blob/main/examples/stem_adv.py
* https://stem.torproject.org/tutorials/to_russia_with_love.html#custom-path-selection
* https://stem.torproject.org/api.html

In [ ]:
def get_geolocation(ip_address):
    url = f"https://ipinfo.io/{ip_address}/json"
    response = requests.get(url)
    data = response.json()

    print(f"IP Address: {ip_address}")
    print(f"Location: {data.get('city', '')}, {data.get('region', '')}, {data.get('country', '')}")
    print(f"ISP: {data.get('org', '')}")
    print("=" * 30)

    return data

In [ ]:
def get_circuit_data(controller, circuit):
    guard_ip, middle_ip, exit_ip = None, None, None
    print("Circuit characteristics:", circuit)
    if circuit.status == CircStatus.BUILT and len(circuit.path) >=3:
        guard_fingerprint = circuit.path[0][0]  # The first hop is the guard relay
        print("GUARD", circuit.path[0])
        middle_fingerprint = circuit.path[1][0]
        print("MIDDLE", circuit.path[1])
        exit_fingerprint = circuit.path[2][0]
        print("LAST", circuit.path[2])

        # Get relay details for each hop
        guard_relay = controller.get_network_status(guard_fingerprint)
        middle_relay = controller.get_network_status(middle_fingerprint)
        exit_relay = controller.get_network_status(exit_fingerprint)

        print("Guard relay flags", guard_relay.flags)
        print("Middle relay flags", middle_relay.flags)
        print("Exit relay flags", exit_relay.flags)

        guard_ip =  guard_relay.address
        middle_ip = middle_relay.address
        exit_ip = exit_relay.address

    else:
        print("Circuit skipped")

    return guard_ip, middle_ip, exit_ip

In [ ]:
def change_permissions_onion_service_folder():
    command = f"chmod 700 {ONION_SERVICE_FOLDER}"
    tor_process = subprocess.Popen(shlex.split(command))
    return tor_process

In [ ]:
def log_callback(line):
    print(line)

In [ ]:
app = Flask(__name__)

@app.route('/')
def index():
    return "<h1>Hi Grandma!</h1>"

def start_web_app():
    app.run(port=WEB_PORT)

### How to create circuits with onion services
* https://gist.github.com/PaulSec/ec8f1689bfde3ce9a920

In [ ]:
def create_dummy_onion_service():

    return stem.process.launch_tor_with_config(
        config={
            'SocksPort': str(SOCKS_PORT),
            'ControlPort': str(CONTROL_PORT),
            'HiddenServiceDir': {ONION_SERVICE_FOLDER},
            'HiddenServicePort': '80 127.0.0.1:8080',
            'Log': 'NOTICE stdout'  # Redirect log messages to stdout
        },
        completion_percent=100,
        init_msg_handler=log_callback
    )

### Getting Tor descriptors
* https://stem.torproject.org/api/descriptor/remote.html

In [ ]:
def print_relay_fingerprints():
    downloader = stem.descriptor.remote.DescriptorDownloader(
        use_mirrors = True,
        timeout = 10,
    )

    query = downloader.get_server_descriptors()

    for desc in query.run():
        print('  %s (%s)' % (desc.nickname, desc.fingerprint))

In [ ]:
def get_onion_service_introduction_points(controller, onion_url):
    try:
        onion_name = onion_url.split('.onion')[0]
        print("onion_name", onion_name)
        desc = controller.get_hidden_service_descriptor(onion_name)

        print(f"{onion_url}'s introduction points are...")
        print("desc", desc)
        for introduction_point in desc.introduction_points():
            print('  %s:%s => %s' % (introduction_point.address, introduction_point.port, introduction_point.identifier))
                
        return controller.get_network_statuses([onion_url])[0].fingerprint
    except Exception as e:
        print("Error getting onion service fingerprint:", e)
        return None

In [ ]:
def get_onion_url():
    with open(ONION_SERVICE_FOLDER+'hostname') as f:
        onion_url = f.readline()
    onion_url = onion_url.split('\n')[0]
    return onion_url

In [ ]:
# def start_onion_service_app():
#     # Start the flask web app in a separate thread
#     t = threading.Thread(target=start_web_app)
#     t.daemon = True
#     t.start()

#     with open(ONION_SERVICE_FOLDER+'hostname') as f:
#         onion_url = f.readlines()
#     print(f"Started serving at {onion_url}")

In [ ]:
# Retries this block 5 times, interleaved by 10000 miliseconds
#@retry(wait_exponential_multiplier=1000, wait_exponential_max=10000, stop_max_attempt_number=5)
def create_new_circuit(controller, client_id, onion_fingerprint):
    print(f"RETRY create_new_circuit({client_id})")

    #onion_url = create_dummy_onion_service(controller)
    #print("onion_url", onion_url)
    print("controller.get_circuits()[-1] BEFORE", controller.get_circuits()[-1])
    circuit_id = controller.new_circuit(await_build=True, timeout=60)
    print("--- circuit_id", circuit_id)
    print("controller.get_circuits()[-1] AFTER", controller.get_circuits()[-1])
    #circuit_id = controller.extend_circuit([circuit_id])
    circuit_id = controller.extend_circuit(circuit_id, [onion_fingerprint])
    print("circuit_id", circuit_id)
    circuit_idx = int(circuit_id) - 1
    print("circuit_idx", circuit_idx)
    #circuit = controller.get_circuits()[circuit_idx]
    #print("new circuit", circuit)
    circuit = controller.get_circuits()[-1]

    # Retrieve the guard nodes
    guard_nodes = controller.get_network_statuses([controller.get_circuit(circuit_id).path[0]])[0]
    onion_service_guard_nodes = [guard_nodes.fingerprint]
    print("Guard Nodes (Onion Service):", onion_service_guard_nodes)

    client_guard_nodes = [controller.get_conf("EntryNodes")[0]]
    print("Guard Nodes (Client):", client_guard_nodes)

    print("new circuit 2", circuit)
    guard_ip, middle_ip, exit_ip = get_circuit_data(controller, circuit)
    get_geolocation(guard_ip)
    get_geolocation(middle_ip)
    get_geolocation(exit_ip)

In [ ]:

def generate_client_circuits(num_clients):
    # Has to be Control Port
    with Controller.from_port() as controller:
    #with Controller.from_port(address="127.0.0.1", port=CONTROL_PORT) as controller:
        controller.authenticate()
        print("Successfully authenticated with Tor control port")

        hidden_service_dir = 'onion_service2'

        # Create a hidden service where visitors of port 80 get redirected to local
        # port 5000 (this is where Flask runs by default).

        print(" * Creating our hidden service in %s" % hidden_service_dir)
        result = controller.create_hidden_service(hidden_service_dir, 80, target_port=WEB_PORT)

        if result is None:
            print("Could not create hidden service. Exiting ...")
            sys.exit(0)

        # The hostname is only available when we can read the hidden service
        # directory. This requires us to be running with the same user as tor.
        onion_url = None
        if result.hostname:
            print(" * Our service is available at %s, press ctrl+c to quit" % result.hostname)
            onion_url = result.hostname
        else:
            print(" * Unable to determine our service's hostname, probably due to being unable to read the hidden service directory")

        try:
            # Starting onion service app as a background thread
            t = threading.Thread(target=start_web_app)
            t.daemon = True
            t.start()
            
            #time.sleep(100)

            print(f"Connecting to onion url {onion_url}")
            onion_fingerprint = get_onion_service_introduction_points(controller, onion_url)
            print(f"Onion fingerprint {onion_fingerprint}")

            # for client_id in range(num_clients):
            #     print("=== Client {}".format(client_id))
            #     create_new_circuit(controller, client_id, onion_fingerprint)
        finally:
            # Shut down the hidden service and clean it off disk. Note that you *don't*
            # want to delete the hidden service directory if you'd like to have this
            # same *.onion address in the future.
            print(" * Shutting down our hidden service")
            controller.remove_hidden_service(hidden_service_dir)
            shutil.rmtree(hidden_service_dir)
        
        
        #except stem.CircuitExtensionFailed as e:
            
        # except Exception as e:
        #     traceback.print_exc()

        # finally:
        #     controller.signal("SHUTDOWN") # Stops the Tor process gracefully
        #     print("Exited Tor process cleanly 2")
    

### Check if Tor is already running
First, restart the kernel. Then, run the following commands:
```
sudo lsof -i -P | grep LISTEN | grep 9050
sudo kill -9 95441
```

In [ ]:
#stop_tor()
# Start Tor process in the background
#tor_process = start_tor()
#change_permissions_onion_service_folder()
#onion_process = None
# try:
#     onion_process = create_dummy_onion_service()
#     print("onion_process", onion_process)
#     print("Tor process started. Waiting for a moment...")
#     time.sleep(10)  # Allow Tor to start
#     #time.sleep(500)  # Allow Tor to start

# except Exception as e:
#     traceback.print_exc()
#     if onion_process:
#         onion_process.terminate()
#         print("Exited Tor process cleanly after error")

# try:
num_clients = 10
generate_client_circuits(num_clients)

# except Exception as e:
#     traceback.print_exc()

# finally:
#     if onion_process:
#         onion_process.terminate()
#         print("Exited Tor process cleanly at the end")

Successfully authenticated with Tor control port
 * Creating our hidden service in /Users/danielalopes/Library/Application Support/TorBrowser-Data/Tor/hello_world
Could not create hidden service. Exiting ...


AttributeError: 'NoneType' object has no attribute 'hostname'

: 